In [1]:
pip install changeforest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
sys.path.append('/content/drive/MyDrive/DCDP/mean')

Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from cpd_utils import *
import changeforest

import time
import bisect

import pandas as pd

# ChangeForest

In [13]:
T = 4
Delta = 50
p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[15.8113883 15.8113883 15.8113883]


In [7]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]

    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 3.59, avg time: 0.25719197988510134
std loc error: 10.099599001940621, std time: 0.014041741081299975
K < K*: 0, K = K*: 84, K > K*: 16


In [8]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)

In [15]:
T = 4
Delta = 50
p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[3.16227766 3.16227766 3.16227766]


In [16]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]
    
    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 4.38, avg time: 0.35381388902664185
std loc error: 10.12894861276332, std time: 0.05296745831930022
K < K*: 0, K = K*: 81, K > K*: 19


In [17]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)

In [22]:
T = 4
Delta = 50
p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 0.5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[1.58113883 1.58113883 1.58113883]


In [23]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]
    
    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

ix = loc_error_rf < np.infty
loc_error_ = loc_error_rf[ix]
run_time_ = run_time_rf[ix]
K_ = K_rf[ix]
print('---------- change forest - random forest - dropna -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_.mean(), run_time_.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_.std(), run_time_.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_ < T - 1), sum(K_ == T - 1), sum(K_ > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 7.15, avg time: 0.37332807540893553
std loc error: 9.569090865907794, std time: 0.07512038243065511
K < K*: 1, K = K*: 78, K > K*: 21
---------- change forest - random forest - dropna -----------
avg loc error: 7.15, avg time: 0.37332807540893553
std loc error: 9.569090865907794, std time: 0.07512038243065511
K < K*: 1, K = K*: 78, K > K*: 21


In [24]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)

In [27]:
T = 4
Delta = 50
p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[15.8113883 15.8113883 15.8113883]


In [28]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]

    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 2.85, avg time: 1.1116501212120056
std loc error: 7.498499849969993, std time: 0.15383033361238993
K < K*: 0, K = K*: 85, K > K*: 15


In [ ]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)

In [30]:
T = 4
Delta = 50
p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[3.16227766 3.16227766 3.16227766]


In [31]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]
    
    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 3.28, avg time: 1.3158190250396729
std loc error: 7.005826146858056, std time: 0.06791649515390792
K < K*: 0, K = K*: 85, K > K*: 15


In [ ]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)

In [35]:
T = 4
Delta = 200
p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 0.5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[1.58113883 1.58113883 1.58113883]


In [36]:
np.random.seed(0)

nt = Delta * T
B = 100

run_time_rf = np.zeros(B)
loc_error_rf = np.zeros(B)
K_rf = np.zeros(B)
cp_best_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    Y_all = np.zeros((2 * nt, p))
    Y_all[np.arange(0, 2 * nt, 2), :] = Y_train
    Y_all[np.arange(1, 2 * nt, 2), :] = Y_test

    cp_truth = cp_truth_list[b]
    
    start_time = time.time()
    result = changeforest.changeforest(Y_all, "random_forest", "bs")
    cp_best = result.split_points()
    cp_best = [x // 2 for x in cp_best]
    
    run_time_rf[b] = time.time() - start_time
    loc_error_rf[b] = cp_distance(cp_best, cp_truth)
    K_rf[b] = len(cp_best)

    cp_best_list.append(cp_best)
    print(b)

print('---------- change forest - random forest -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_rf.mean(), run_time_rf.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_rf.std(), run_time_rf.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_rf < T - 1), sum(K_rf == T - 1), sum(K_rf > T - 1)))

ix = loc_error_rf < np.infty
loc_error_ = loc_error_rf[ix]
run_time_ = run_time_rf[ix]
K_ = K_rf[ix]
print('---------- change forest - random forest - dropna -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_.mean(), run_time_.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_.std(), run_time_.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_ < T - 1), sum(K_ == T - 1), sum(K_ > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- change forest - random forest -----------
avg loc error: 14.73, avg time: 5.494371454715729
std loc error: 30.498804894618413, std time: 0.31395854893866476
K < K*: 0, K = K*: 82, K > K*: 18
---------- change forest - random forest - dropna -----------
avg loc error: 14.73, avg time: 5.494371454715729
std loc error: 30.498804894618413, std time: 0.31395854893866476
K < K*: 0, K = K*: 82, K > K*: 18


In [ ]:
import pickle
with open(path + 'rf_bs_n' + str(nt) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, loc_error_rf, run_time_rf, K_rf], f)